# 贷款违约预测

## 一、导入数据

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',100)
pd.set_option('expand_frame_repr',False)

In [2]:
train_df=pd.read_csv(r'D:\文件\学习\数据集\【天池】贷款违约预测\train.csv')
test_df=pd.read_csv(r'D:\文件\学习\数据集\【天池】贷款违约预测\testA.csv')

## 二、统计分析

### 字段说明(Field Description)

<br>id	为贷款清单分配的唯一信用证标识
<br>loanAmnt	贷款金额
<br>term	贷款期限（year）
<br>interestRate	贷款利率
<br>installment	分期付款金额
<br>grade	贷款等级
<br>subGrade	贷款等级之子级
<br>employmentTitle	就业职称
<br>employmentLength	就业年限（年）
<br>homeOwnership	借款人在登记时提供的房屋所有权状况
<br>annualIncome	年收入
<br>verificationStatus	验证状态
<br>issueDate	贷款发放的月份
<br>purpose	借款人在贷款申请时的贷款用途类别
<br>postCode	借款人在贷款申请中提供的邮政编码的前3位数字
<br>regionCode	地区编码
<br>dti	债务收入比
<br>delinquency_2years	借款人过去2年信用档案中逾期30天以上的违约事件数
<br>ficoRangeLow	借款人在贷款发放时的fico所属的下限范围
<br>ficoRangeHigh	借款人在贷款发放时的fico所属的上限范围
<br>openAcc	借款人信用档案中未结信用额度的数量
<br>pubRec	贬损公共记录的数量
<br>pubRecBankruptcies	公开记录清除的数量
<br>revolBal	信贷周转余额合计
<br>revolUtil	循环额度利用率，或借款人使用的相对于所有可用循环信贷的信贷金额
<br>totalAcc	借款人信用档案中当前的信用额度总数
<br>initialListStatus	贷款的初始列表状态
<br>applicationType	表明贷款是个人申请还是与两个共同借款人的联合申请
<br>earliesCreditLine	借款人最早报告的信用额度开立的月份
<br>title	借款人提供的贷款名称
<br>policyCode	公开可用的策略_代码=1新产品不公开可用的策略_代码=2
<br>n系列匿名特征	匿名特征n0-n14，为一些贷款人行为计数特征的处理

In [3]:
train_df.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,110000.0,2,2014-07-01,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,Aug-2001,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,46000.0,2,2012-08-01,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,May-2002,1723.0,1.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,74000.0,2,2015-10-01,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,May-2006,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,118000.0,1,2015-08-01,0,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,May-1999,4.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,29000.0,2,2016-03-01,0,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,Aug-1977,11.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


In [33]:
train_df.shape

(800000, 47)

## 三、数据清洗

### （一）区分数据类型

In [4]:
#查看定类数据情况
#grade,subGrad虽然是定类数据，但是同时也是定序数据，字母代表的含义有不同的重要性等级

In [5]:
#grade,gradeA是等级最高的
train_df['grade'].value_counts()

B    233690
C    227118
A    139661
D    119453
E     55661
F     19053
G      5364
Name: grade, dtype: int64

In [6]:
#复制一份数据
train_data=train_df.copy()
test_data=test_df.copy()

In [7]:
#grade

In [8]:
#使用LabelEncoder对数据进行变换
from sklearn.preprocessing import LabelEncoder
labelencoder_=LabelEncoder()
#训练集
scaler_train_grade=labelencoder_.fit_transform(train_data[['grade']])
train_data['grade']=scaler_train_grade                                     
#测试集，测试集只要transform就行
scaler_test_grade=labelencoder_.transform(test_data[['grade']])
test_data['grade']=scaler_test_grade

In [9]:
#subGrade

In [10]:
#使用LabelEncoder对subGrade进行变换
#训练集
LabelEncode_subGrade=LabelEncoder()
scaler_train_subgrade=LabelEncode_subGrade.fit_transform(train_data[['subGrade']])
train_data['subGrade']=scaler_train_subgrade
#测试集
scaler_test_subgrade=LabelEncode_subGrade.transform(test_data[['subGrade']])
test_data['subGrade']=scaler_test_subgrade

In [11]:
train_data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,4,21,320.0,2 years,2,110000.0,2,2014-07-01,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,Aug-2001,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.90,3,16,219843.0,5 years,0,46000.0,2,2012-08-01,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,May-2002,1723.0,1.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000.0,5,16.99,298.17,3,17,31698.0,8 years,0,74000.0,2,2015-10-01,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,May-2006,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,0,3,46854.0,10+ years,1,118000.0,1,2015-08-01,0,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,May-1999,4.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,2,11,54.0,NaN,1,29000.0,2,2016-03-01,0,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,Aug-1977,11.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


In [12]:
test_data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,800000,14000.0,3,10.99,458.28,1,7,7027.0,10+ years,0,80000.0,0,2014-07-01,0,163.0,21,10.56,1.0,715.0,719.0,17.0,0.0,0.0,9846.0,30.7,29.0,0,0,Nov-1974,0.0,1.0,1.0,4.0,6.0,6.0,6.0,8.0,4.0,15.0,19.0,6.0,17.0,0.0,0.0,1.0,3.0
1,800001,20000.0,5,14.65,472.14,2,14,60426.0,10+ years,0,50000.0,0,2015-07-01,2,235.0,8,21.40,2.0,670.0,674.0,5.0,0.0,0.0,8946.0,56.6,14.0,0,0,Jul-2001,5.0,1.0,2.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,9.0,3.0,5.0,0.0,0.0,2.0,2.0
2,800002,12000.0,3,19.99,445.91,3,18,23547.0,2 years,1,60000.0,2,2016-10-01,0,526.0,20,33.50,0.0,710.0,714.0,12.0,0.0,0.0,970.0,17.6,43.0,1,0,Aug-2006,0.0,1.0,0.0,1.0,4.0,4.0,1.0,1.0,36.0,5.0,6.0,4.0,12.0,0.0,0.0,0.0,7.0
3,800003,17500.0,5,14.31,410.02,2,13,636.0,4 years,0,37000.0,1,2014-11-01,4,248.0,11,13.95,0.0,685.0,689.0,10.0,1.0,1.0,10249.0,52.3,18.0,0,0,Jul-2002,4.0,1.0,0.0,2.0,2.0,2.0,4.0,7.0,2.0,8.0,14.0,2.0,10.0,0.0,0.0,0.0,3.0
4,800004,35000.0,3,17.09,1249.42,3,15,368446.0,< 1 year,1,80000.0,1,2017-10-01,0,115.0,8,24.97,0.0,685.0,689.0,19.0,0.0,0.0,33199.0,35.6,22.0,0,0,Dec-2000,0.0,1.0,0.0,8.0,11.0,11.0,9.0,11.0,3.0,16.0,18.0,11.0,19.0,0.0,0.0,0.0,1.0


In [13]:
train_data['employmentLength'].value_counts()

10+ years    262753
2 years       72358
< 1 year      64237
3 years       64152
1 year        52489
5 years       50102
4 years       47985
6 years       37254
8 years       36192
7 years       35407
9 years       30272
Name: employmentLength, dtype: int64

In [14]:
#employmentLength

In [15]:
#将employmentLength进行数字化
#定义employmentLength的函数
def label_to_number(df):
    df['employmentLength']=df['employmentLength'].replace('1 year','1')
    df['employmentLength']=df['employmentLength'].replace('2 years','2')
    df['employmentLength']=df['employmentLength'].replace('3 years','3')
    df['employmentLength']=df['employmentLength'].replace('4 years','4')
    df['employmentLength']=df['employmentLength'].replace('5 years','5')
    df['employmentLength']=df['employmentLength'].replace('6 years','6')
    df['employmentLength']=df['employmentLength'].replace('7 years','7')
    df['employmentLength']=df['employmentLength'].replace('8 years','8')
    df['employmentLength']=df['employmentLength'].replace('9 years','9')
    df['employmentLength']=df['employmentLength'].replace('10+ years','10')
    df['employmentLength']=df['employmentLength'].replace('< 1 year','0')
    return df

In [16]:
#调用方法转换employmentLength的缺失值
train_data=label_to_number(train_data)

test_data=label_to_number(test_data)

In [17]:
#填补employmentLength的缺失值
#查看训练集上的employmentLength的众数,可以发现缺失值要用10填补
train_data['employmentLength'].value_counts()

train_data['employmentLength']=train_data['employmentLength'].fillna(10)
test_data['employmentLength']=test_data['employmentLength'].fillna(10)

In [18]:
#对issueDate转换成日期时间格式
def transform_datetime(df):
    from datetime import datetime
    df['issueDate']=df['issueDate'].str.split('-')
    df['issue_year']=df['issueDate'].str.get(0)
    df['issue_month']=df['issueDate'].str.get(1)
    df['issue_day']=df['issueDate'].str.get(2)
    return df

In [19]:
#调用日期转换函数
train_data=transform_datetime(train_data)
test_data=transform_datetime(test_data)

In [20]:
##对earliesCreditLine进行变换
def transform_CreditLine(df):
    df['earliesCreditLine']=df['earliesCreditLine'].str.split('-')
    df['earliesCreditLine_year']=df['earliesCreditLine'].str.get(1)
    df['earliesCreditLine_month']=df['earliesCreditLine'].str.get(0)
    return df

In [21]:
#调用earliesCreditLine函数
train_data=transform_CreditLine(train_data)
test_data=transform_CreditLine(test_data)

In [22]:
#定义用众数填充的函数
def fillna_with_mode(df,col_name):
    col_val_counts=df[col_name].value_counts()
    df[col_name]=df[col_name].fillna(col_val_counts.index[0])
    return df

In [23]:
#填补employmentTitle的缺失值
train_data=fillna_with_mode(train_data,'employmentTitle')

In [24]:
#填补postCode，用众数
train_data=fillna_with_mode(train_data,'postCode')

In [25]:
#pubRecBankruptcies填补，用众数
train_data=fillna_with_mode(train_data,'pubRecBankruptcies')
test_data=fillna_with_mode(test_data,'pubRecBankruptcies') #测试集合也用训练集合的众数填充

In [26]:
#revolUtil填补，用众数
train_data=fillna_with_mode(train_data,'revolUtil')
test_data=fillna_with_mode(test_data,'revolUtil')

In [27]:
#title填补，用众数
train_data=fillna_with_mode(train_data,'title')
test_data=fillna_with_mode(test_data,'title')

In [28]:
#用众数填补n0~n14
for i in range(0,15):
    train_data=fillna_with_mode(train_data,'n{}'.format(i))
    test_data=fillna_with_mode(test_data,'n{}'.format(i))

In [29]:
#dti用均值填充
train_data['dti']=train_data['dti'].fillna(train_data['dti'].mean())
test_data['dti']=test_data['dti'].fillna(train_data['dti'].mean())

In [30]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       800000 non-null  int64  
 1   loanAmnt                 800000 non-null  float64
 2   term                     800000 non-null  int64  
 3   interestRate             800000 non-null  float64
 4   installment              800000 non-null  float64
 5   grade                    800000 non-null  int32  
 6   subGrade                 800000 non-null  int32  
 7   employmentTitle          800000 non-null  float64
 8   employmentLength         800000 non-null  object 
 9   homeOwnership            800000 non-null  int64  
 10  annualIncome             800000 non-null  float64
 11  verificationStatus       800000 non-null  int64  
 12  issueDate                800000 non-null  object 
 13  isDefault                800000 non-null  int64  
 14  purp

In [31]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 51 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       200000 non-null  int64  
 1   loanAmnt                 200000 non-null  float64
 2   term                     200000 non-null  int64  
 3   interestRate             200000 non-null  float64
 4   installment              200000 non-null  float64
 5   grade                    200000 non-null  int32  
 6   subGrade                 200000 non-null  int32  
 7   employmentTitle          200000 non-null  float64
 8   employmentLength         200000 non-null  object 
 9   homeOwnership            200000 non-null  int64  
 10  annualIncome             200000 non-null  float64
 11  verificationStatus       200000 non-null  int64  
 12  issueDate                200000 non-null  object 
 13  purpose                  200000 non-null  int64  
 14  post

In [32]:
train_data.to_csv(r'D:\文件\学习\数据集\【天池】贷款违约预测\train_afterCleaning.csv')
test_data.to_csv(r'D:\文件\学习\数据集\【天池】贷款违约预测\test_afterCleaning.csv')